In [ ]:
import numpy as np
import pandas as pd
import tqdm
import re

In [ ]:
from io import BytesIO
from PIL import Image
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds

In [ ]:
Int = pd.read_csv('Interactions-3.csv')
#items = pd.read_csv("items.csv")
items = pd.read_csv("items_with_text.csv")

In [ ]:
items = items.groupby(["picture_url"]).head(1)[["text", "picture_url"]]

In [ ]:
Int = Int.merge(items[["picture_url", "text"]].drop_duplicates(), on=["picture_url"])

In [ ]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def rec_imaging(product_ids, content_dict, measure=None, top_n=5):
    picture_urls = [content_dict[i] for i in product_ids]
    for idx, chunk in enumerate(chunks(picture_urls, top_n)):
        fig = plt.figure(figsize=(20, 4))
        for n, i in enumerate(chunk):
            try:
                r = requests.get(i)
                im = Image.open(BytesIO(r.content))

            except:
                print('Something went wrong with {url}'.format(url=i))
                continue

            a = fig.add_subplot(1, top_n, n + 1)
            if measure is not None:
                a.title.set_text("measure = {}".format(np.round(measure[idx], 4)))
            plt.imshow(im)
            plt.axis('off')
        plt.show()

# Холодный старт

# Самые популярные

In [ ]:
class Top():
    

In [ ]:
T = Top(Int)

In [ ]:
text = T.top_items()

In [ ]:
text[:8]

# Cовстречаемость

In [ ]:
class Recomendations():
    def __init__(self, Int):
        self.cooc_rec = None
        self.product_id_to_url = {}
        for i in Int[["product_id", "picture_url"]].drop_duplicates().values:
            self.product_id_to_url[i[0]] = i[1]
        self.interactions = Int
        
    def coocurrency_count(self):
        # считаем веса для разных типов актвиности


In [ ]:
cooc_rec = Recomendations(Int)
cooc_rec.coocurrency_count()

In [ ]:
cooc_rec.get_rec(10)

In [ ]:
cooc_rec.get_rec(121)

In [ ]:
cooc_rec.get_rec(324)

In [ ]:
cooc_rec.get_rec(1111)

In [ ]:
cooc_rec.get_rec(1236)

In [ ]:
cooc_rec.get_rec(12362)

In [ ]:
cooc_rec.get_rec(1002)

In [ ]:
cooc_rec.get_rec(1245)

In [ ]:
cooc_rec.get_rec(1247)

In [ ]:
cooc_rec.get_rec(3453)

In [ ]:
cooc_rec.get_rec(3445)

In [ ]:
cooc_rec.get_rec(667)

In [ ]:
cooc_rec.get_rec(1231)

In [ ]:
cooc_rec.get_rec(12326)

In [ ]:
cooc_rec.get_rec(12341)

In [ ]:
cooc_rec.get_rec(300)

# Content-based

In [ ]:
class Content_Based():
    def __init__(self, items, interactions):
        self.items = items
        self.interactions = interactions
        self.content_dict = {}
        for i, j in enumerate(items["picture_url"]):
            self.content_dict[i] = j
        self.inversed_dict = {v: k for k, v in self.content_dict.items()}
        


In [ ]:
CB = Content_Based(items, Int)

In [ ]:
CB.get_items_representation()

In [ ]:
CB.get_rec_I2I(13)

In [ ]:
CB.get_rec(1325)

In [ ]:
CB.get_rec(3131)

In [ ]:
CB.get_rec(512)

In [ ]:
CB.get_rec(1234)

In [ ]:
CB.get_rec(3212)

In [ ]:
CB.get_rec(333)

In [ ]:
CB.get_rec_U2I(112)

# W2V recommendations

In [ ]:
class W2V_REC():
    def __init__(self, interactions):
        self.interactions = interactions
        self.interactions["product_id"] = self.interactions["product_id"].astype(str)
        self.content_dict = {}
        for i, j in interactions[["product_id", "picture_url"]].drop_duplicates().values:
            self.content_dict[i] = j



In [ ]:
w = W2V_REC(Int)
w.get_w2v()

In [ ]:
w.get_rec_I2I("0")

In [ ]:
w.get_rec_I2I("11111")

In [ ]:
w.get_rec_U2I(2122)

In [ ]:
user = 111
cont_based.get_rec_for_user(user)
w.get_rec_U2I(user)

# Colloborative filtering

In [ ]:
class Colloborative():
    def __init__(self, interactions):
        self.content_dict = {}
        for i, j in interactions[["product_id", "picture_url"]].drop_duplicates().values:
            self.content_dict[i] = j
        df_weights = pd.DataFrame({"page_type": ["PRODUCT", "CART", "PURCHASE"],
                                   'weight': [1, 1, 1]})
        weighted = interactions.merge(df_weights, on="page_type")\
                          .groupby(["vid", "product_id", "picture_url"])["weight"]\
                          .sum()\
                          .reset_index()
        weighted["weight"] = (weighted["weight"] > 0).astype(int)
        self.csr_rates = coo_matrix((weighted["weight"], (weighted["vid"], weighted["product_id"])), 
                            shape=(len(set(weighted["vid"])), len(set(weighted["product_id"]))))
        

In [ ]:
clb = Colloborative(Int)

In [ ]:
clb.item_based(0)

In [ ]:
clb.item_based(12)

In [ ]:
clb.item_based(123)

In [ ]:
clb.item_based(1234)

In [ ]:
clb.item_based(31211)

In [ ]:
clb.item_based(2556)

In [ ]:
clb.user_based(0)

In [ ]:
clb.user_based(123)

In [ ]:
clb.user_based(12345)

In [ ]:
clb.user_based(123456)

In [ ]:
clb.user_based(34434)

In [ ]:
clb.user_based(3333)

In [ ]:
clb.user_based(5555)

In [ ]:
clb.user_based(77777)

In [ ]:
clb.user_based(5)

In [ ]:
clb.user_based(323)

# Matrix Factorization

In [ ]:
Int.head()

In [ ]:
class MatrixFactorization():
    
    def __init__(self, interactions, k):
        interactions["product_id"] = interactions["product_id"].astype(int)
        self.k = k
        self.content_dict = {}
        for i, j in interactions[["product_id", "picture_url"]].drop_duplicates().values:
            self.content_dict[i] = j
        df_weights = pd.DataFrame({"page_type": ["PRODUCT", "CART", "PURCHASE"],
                                   'weight': [1, 1, 1]})
        weighted = interactions.merge(df_weights, on="page_type")\
                          .groupby(["vid", "product_id", "picture_url"])["weight"]\
                          .sum()\
                          .reset_index()
        weighted["weight"] = (weighted["weight"] > 0).astype(int)
        self.csr_rates = coo_matrix((weighted["weight"], (weighted["vid"], weighted["product_id"])), 
                            shape=(len(set(weighted["vid"])), len(set(weighted["product_id"]))))
        


In [ ]:
simple_svd = MatrixFactorization(Int, 30)
simple_svd.MF()

In [ ]:
simple_svd.get_rec(10)

In [ ]:
simple_svd.get_rec(1002)

In [ ]:
simple_svd.get_rec(10022)

In [ ]:
simple_svd.get_rec(12342)

# LightFM

In [ ]:
mf = MatrixFactorization(Int, 300)
mf.LightFM()

In [ ]:
simple_svd.get_rec(123)

In [ ]:
mf.get_rec(123)

In [ ]:
simple_svd.get_rec(1234)

In [ ]:
mf.get_rec(1234)